# GR5293 - Proj2 - Group9
## NLP with tweets related to COVID
#### NLP pipeline with sentiment prediction
* Tokenization
    > Split text into tokens(sentences or words), for this question, we split the document into sentence for automatic summarization, and words for sentiment analysis and topic modeling
* Screen out stop words and other meaningless corpus
* Lemmatization
    > Here we only use lemmatization rather than stemming is because lemmatization keeps the interpretability of words with their context. While stemming might lead to incorrect meaning. It is important to make morphological analysis of the words. 
* EDA: wordCloud with different sentiment
    > Identify what poeple with different emotions were considering about
* EDA: Word2vec with Clustering
    > Word2Vec: Effective for detecting the synonymous words or suggesting additional words for a partial sentence

    Clustering methods: K-means + DBScan

    Use all the words in a specific part-of-speech from all the documents (e.g. all nouns / all adj.s)
* (word2vec w/ recommendation)?
* Topic Modeling: Feature extraction by TFIDF + Latent Dirichlet Allocation
    Build a pipeline with KFoldCV to find the best topic number
* Automatic summrization
    > Identify what were most people thinking about or tweeting for
* Sentiment Analysis: Classification for sentiment(5 classes: Neutral / Positive / Extremely Positive / Negative / Extremely Negative)
    Potential Model: BERT?
#### Preprocessing
* One-hot encoding

In [2]:
import numpy
import numpy as np
import pandas as pd
import sklearn
import nltk
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import re
from sklearn import pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
import xgboost
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
os.getcwd()

'/Users/kangshuoli/Documents/VScode_workspace/GR5293/EODS-Project2-Group9/EODS_Project2_Group9/doc'

### Read in filtered data

In [4]:
df = pd.read_csv("../data/Corona_NLP_filtered.csv")
df.drop(
    "Unnamed: 0", 
    axis = 1,
    inplace = True
)
df

,OriginalTweet,Sentiment,Tweet_filtered,Word_list,Senten_list
0,advice Talk to your neighbours family to excha...,Positive,advice Talk to your neighbours family to excha...,"['advice', 'talk', 'neighbour', 'family', 'exc...",['advice Talk to your neighbours family to exc...
1,Coronavirus Australia: Woolworths to give elde...,Positive,Coronavirus Australia: Woolworths to give elde...,"['coronavirus', 'australia', 'woolworth', 'giv...",['Coronavirus Australia: Woolworths to give el...
2,My food stock is not the only one which is emp...,Positive,My food stock is not the only one which is emp...,"['food', 'stock', 'one', 'empty', 'please', 'd...",['My food stock is not the only one which is e...
3,"Me, ready to go at supermarket during the #COV...",Extremely Negative,"Me, ready to go at supermarket during the #COV...","['ready', 'go', 'supermarket', 'covid', 'outbr...","['Me, ready to go at supermarket during the #C..."
4,As news of the regionÂs first confirmed COVID...,Positive,As news of the regionÂs first confirmed COVID...,"['news', 'regionâ', '\x92', 'first', 'confirme...",['As news of the regionÂ\x92s first confirmed ...
...,...,...,...,...,...
44250,Meanwhile In A Supermarket in Israel -- People...,Positive,Meanwhile In A Supermarket in Israel -- People...,"['meanwhile', 'supermarket', 'israel', 'people...",['Meanwhile In A Supermarket in Israel -- Peop...
44251,Did you panic buy a lot of non-perishable item...,Negative,Did you panic buy a lot of non-perishable item...,"['panic', 'buy', 'lot', 'nonperishable', 'item...",['Did you panic buy a lot of non-perishable it...
44252,Asst Prof of Economics @cconces was on @NBCPhi...,Neutral,Asst Prof of Economics was on talking about he...,"['asst', 'prof', 'economics', 'talking', 'rece...","[""Asst Prof of Economics @cconces was on @NBCP..."
44253,Gov need to do somethings instead of biar je r...,Extremely Negative,Gov need to do somethings instead of biar je r...,"['gov', 'need', 'somethings', 'instead', 'biar...","[""Gov need to do somethings instead of biar je..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44255 entries, 0 to 44254
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   OriginalTweet   44255 non-null  object
 1   Sentiment       44251 non-null  object
 2   Tweet_filtered  44249 non-null  object
 3   Word_list       44249 non-null  object
 4   Senten_list     44249 non-null  object
dtypes: object(5)
memory usage: 1.7+ MB


### Data Cleaning
Done in 

In [ ]:
def remove_urls(text):
    url_remove = re.compile(r'https?://\S+|www\.\S+')
    return url_remove.sub(r'', text)

def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'', text)

def lower(text):
    low_text = text.lower()
    return low_text

def remove_num(text):
    remove = re.sub(r'\d+', '' ,text)
    return remove

def remove_punctuation(text):
    clean_list = [char for char in text if char not in string.punctuation]
    clean_str = ''.join(clean_list)
    return clean_str

df['OriginalTweet'] = df['OriginalTweet'].apply(lambda x: remove_urls(x)) \
                                               .apply(lambda x: remove_html(x)) \
                                               .apply(lambda x: lower(x)) \
                                               .apply(lambda x: remove_num(x)) \
                                               .apply(lambda x: remove_punctuation(x))


def classes_def(x):
    if x ==  'Extremely Positive' or x == 'Positive':
        return "positive"
    elif x == "Extremely Negative" or x == 'Negative':
        return "negative"
    else:
        return "neutral"
    
train['Sentiment'] = train['Sentiment'].apply(lambda x: classes_def(x))

train.head()

## Word-level Analysis
1. wordCloud
2. word2Vec w/ clustering

## Document-level / Sentence-level Analysis